In [3]:
import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from common import first_prep, get_lags

In [28]:
tv = TvDatafeed()
df = tv.get_hist("DAX", "XETR", Interval.in_30_minute, 5000).reset_index()

In [29]:
data = first_prep(df)
data

,date,open,close,high,low,iblow,ibhigh,ibclose,daydir,ibdir
0,2021-03-10,14439.45,14540.25,14560.97,14408.52,14408.52,14488.46,14477.09,up,up
1,2021-03-11,14553.54,14569.39,14595.02,14515.07,14521.89,14578.46,14528.80,up,down
2,2021-03-12,14492.45,14502.39,14527.42,14450.72,14469.48,14527.42,14484.69,up,down
3,2021-03-15,14504.49,14461.42,14572.24,14409.35,14500.52,14572.24,14544.67,down,up
4,2021-03-16,14519.72,14557.58,14601.79,14511.30,14511.30,14550.42,14550.27,up,up
...,...,...,...,...,...,...,...,...,...,...
272,2022-04-04,14503.96,14518.16,14535.07,14334.10,14334.10,14515.79,14382.35,up,down
273,2022-04-05,14504.80,14424.36,14603.44,14347.33,14471.09,14603.44,14563.51,down,up
274,2022-04-06,14402.16,14151.69,14437.91,14027.18,14216.42,14437.91,14245.02,down,down
275,2022-04-07,14187.13,14078.15,14315.35,14069.53,14123.05,14234.65,14155.22,down,down


In [18]:
import pandas_ta as ta
data.ta

In [34]:
data.ta.chop(append=True)
data.ta.ema(10, append=True)

0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
           ...     
272    14432.734193
273    14431.211612
274    14380.389501
275    14325.436864
276    14317.842889
Name: EMA_10, Length: 277, dtype: float64

In [35]:
data.dropna(inplace=True)

In [37]:
data.rename(columns={"CHOP_14_1_100" : "chop", "EMA_10" : "ema"}, inplace=True)

In [39]:
get_lags(data, "chop", 1)
get_lags(data, "ema", 1)

In [44]:
data.dropna(inplace=True)
def get_pos(a, b):
    if a > b:
        return "above"
    if a == b:
        return "eq"
    return "below"
data["open_ema"] = data.apply(lambda r: get_pos(r.open,r.ema_lag_1), axis=1)
data

,date,open,close,high,low,iblow,ibhigh,ibclose,daydir,ibdir,chop,ema,chop_lag_1,ema_lag_1,open_ema
15,2021-03-31,14982.12,15008.34,15026.49,14973.59,14973.59,15026.49,14991.85,up,up,45.961846,14777.094668,46.449936,14725.706817,above
16,2021-04-01,15053.77,15107.17,15110.92,15032.52,15032.52,15104.14,15088.75,up,up,41.006356,14837.108365,45.961846,14777.094668,above
17,2021-04-06,15289.97,15212.68,15311.86,15209.90,15211.18,15311.86,15231.55,down,down,32.782791,14905.394117,41.006356,14837.108365,above
18,2021-04-07,15221.23,15176.36,15242.73,15160.28,15203.31,15242.73,15221.98,down,up,31.729214,14954.660641,32.782791,14905.394117,above
19,2021-04-08,15236.44,15202.68,15247.78,15144.11,15180.34,15247.78,15181.77,down,down,32.491410,14999.755070,31.729214,14954.660641,above
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,2022-04-04,14503.96,14518.16,14535.07,14334.10,14334.10,14515.79,14382.35,up,down,59.726386,14432.734193,42.534820,14413.750680,above
273,2022-04-05,14504.80,14424.36,14603.44,14347.33,14471.09,14603.44,14563.51,down,up,55.890480,14431.211612,59.726386,14432.734193,above
274,2022-04-06,14402.16,14151.69,14437.91,14027.18,14216.42,14437.91,14245.02,down,down,53.749489,14380.389501,55.890480,14431.211612,below
275,2022-04-07,14187.13,14078.15,14315.35,14069.53,14123.05,14234.65,14155.22,down,down,53.157783,14325.436864,53.749489,14380.389501,below


In [164]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [165]:
dcf = DecisionTreeClassifier()
dcf = dcf.fit(X_train, y_train)

In [166]:
accuracy_score(dcf.predict(X_train), y_train)

1.0

In [167]:
accuracy_score(dcf.predict(X_test), y_test)

0.5619047619047619

In [168]:
cross_val_score(dcf, X_train, y_train, cv=10)

array([0.75      , 0.625     , 0.4375    , 0.8125    , 0.3125    ,
       0.5625    , 0.75      , 0.6       , 0.46666667, 0.66666667])

In [169]:
dcf = dcf.fit(X_train,y_train)
accuracy_score(dcf.predict(X_t)

0.5714285714285714